In [4]:
!pip install tensorflow


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import random
import tensorflow as tf


In [6]:
dataset_path = r"C:\Users\Sridhar\PycharmProjects\per person\CATS & DOGS\DATA\dogs_n_cats-20251105T071022Z-1-001\dogs_n_cats"


In [11]:
cats_dir = os.path.join(dataset_path, "cats")
dogs_dir = os.path.join(dataset_path, "dogs")


In [12]:
print(cats_dir)

C:\Users\Sridhar\PycharmProjects\per person\CATS & DOGS\DATA\dogs_n_cats-20251105T071022Z-1-001\dogs_n_cats\cats


In [13]:
import os
dataset_path = r"C:\Users\Sridhar\PycharmProjects\per person\CATS & DOGS\DATA\dogs_n_cats-20251105T071022Z-1-001\dogs_n_cats"
print(os.listdir(dataset_path))


['.DS_Store', 'single_prediction', 'test_set', 'training_set']


In [21]:
!pip install torch torchvision torchaudio

  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
!pip install torch torchvision torchaudio matplotlib

  Using cached matplotlib-3.10.7-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.3-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.1-cp313-cp313-win_amd64.whl.metadata (114 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-win_amd64.whl.metadata (6.4 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.7-cp313-cp313-win_amd64.whl (8.1 MB)
Using cached contourpy-1.3.3-cp313-cp313-win_amd64.whl (226 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.60.1-cp313-cp313-win_amd64.whl (2.3 MB)
Using cached kiwisolver-1.4.9-cp313-cp313-win_amd64.whl (73 kB)
Using cached pyparsing-3.2.5-py3-none-any.whl (113 kB)

   ---------------------------------------- 0/6 [pyparsing]
   ---------------------------------------- 0/6 [pyparsing]
   ---------------------------------------- 0/6 [pyparsing]
   ------ ---------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import random
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split

# ⃣ Set dataset path
data_dir = r"C:\Users\Sridhar\PycharmProjects\per person\CATS & DOGS\DATA\dogs_n_cats-20251105T071022Z-1-001"

In [25]:
# ⃣ Define transforms
train_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])


In [26]:
#  Load full dataset
full_dataset = datasets.ImageFolder(data_dir)

# ⃣ Organize indices by class
class_idx = {0: [], 1: []}  # 0=cats, 1=dogs
for i, (_, label) in enumerate(full_dataset):
    class_idx[label].append(i)

In [27]:
# ⃣ Select subset (adjust if fewer than 250 images per class)
num_cats = min(250, len(class_idx[0]))
num_dogs = min(250, len(class_idx[1]))
selected = random.sample(class_idx[0], num_cats) + random.sample(class_idx[1], num_dogs)

subset_dataset = Subset(full_dataset, selected)

# Split subset into train/test (80/20)
train_size = int(0.8 * len(subset_dataset))
test_size = len(subset_dataset) - train_size
train_set, test_set = random_split(subset_dataset, [train_size, test_size])

In [28]:
# ⃣ Apply transforms
train_set.dataset.transform = train_transform
test_set.dataset.transform = test_transform

# Create DataLoaders
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

#  Summary
print(f"Total images in subset: {len(subset_dataset)}")
print(f"Train images: {len(train_set)}, Test images: {len(test_set)}")

Total images in subset: 250
Train images: 200, Test images: 50


In [28]:
import os
import random
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# ----------------------------
# Paths to your dataset
# ----------------------------
train_dir = r"C:\Users\Sridhar\PycharmProjects\per person\CATS & DOGS\DATA\dogs_n_cats-20251105T071022Z-1-001\dogs_n_cats\training_set"
test_dir = r"C:\Users\Sridhar\PycharmProjects\per person\CATS & DOGS\DATA\dogs_n_cats-20251105T071022Z-1-001\dogs_n_cats\test_set"

# ----------------------------
# Image parameters
# ----------------------------
img_width, img_height = 64, 64  # smaller size for reduced data
batch_size = 32

# ----------------------------
# Data generators
# ----------------------------
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

# ----------------------------
# Define CNN Model
# ----------------------------
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(68,activation='relu'),Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# ----------------------------
# Fit the model
# ----------------------------
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

# ----------------------------
# Evaluate on test set
# ----------------------------
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='binary',
    shuffle=False
)

loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy*100:.2f}%")


Found 6400 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.
Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 72s 350ms/step - accuracy: 0.6048 - loss: 0.6476 - val_accuracy: 0.6631 - val_loss: 0.6004
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 45s 224ms/step - accuracy: 0.7208 - loss: 0.5441 - val_accuracy: 0.7275 - val_loss: 0.5405
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 27s 134ms/step - accuracy: 0.7714 - loss: 0.4727 - val_accuracy: 0.7581 - val_loss: 0.5080
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 26s 130ms/step - accuracy: 0.8075 - loss: 0.4158 - val_accuracy: 0.7675 - val_loss: 0.5083
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 20s 98ms/step - accuracy: 0.8517 - loss: 0.3289 - val_accuracy: 0.7613 - val_loss: 0.5287
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 18s 87ms/step - accuracy: 0.8972 - loss: 0.2436 - val_accuracy: 0.7450 - val_loss: 0.5876
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 17s 84ms/step - accuracy: 0.9372 - loss: 0.1660 - val_accuracy: 0.7525 - val_loss: 0.7644
Ep